In [1]:
# load required packages
from keras.applications import *
from keras.applications.vgg16 import *
from keras.models import *
from keras.preprocessing import image
from keras.preprocessing.image import *
from keras.layers import Dense,Flatten,GlobalAveragePooling2D,Dropout,BatchNormalization
import h5py as h
import numpy as np
from PIL import Image

import os
import glob
import shutil
from keras import backend as K

from keras.utils import to_categorical
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import math
from utils import *

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotli

In [2]:
train_path = '/home/ec2-user/dogs_cats/train/'
valid_path = '/home/ec2-user/dogs_cats/valid/'
test_path = '/home/ec2-user/dogs_cats/test/'

In [3]:
batch_size =128

In [58]:
#preprocess_data
trn_args = dict(preprocessing_function=resnet50.preprocess_input,
               zoom_range = 0.2, #0.2,0.3
               rotation_range = 20, 
                shear_range = 0.2,
             width_shift_range = 0.1,
           height_shift_range = 0.1,
             horizontal_flip = True,
             channel_shift_range = 0.2
                
               )
test_args = dict(preprocessing_function=resnet50.preprocess_input)
trn_gen = preprocess_data(trn_args)
test_gen = preprocess_data(test_args)

In [59]:
#load data by generator
train_generator = get_data(trn_gen,train_path,input_size=224,batch_size=batch_size)
valid_generator = get_data(test_gen,valid_path,input_size=224,batch_size=batch_size)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [60]:
# one-hot label
trn_label = to_categorical(train_generator.classes,num_classes=2)
val_label = to_categorical(valid_generator.classes,num_classes=2)

In [61]:
# compute steps
trn_steps = comput_step(train_generator.n,batch_size)
val_steps = comput_step(valid_generator.n,batch_size)

In [8]:
# create pre-trained 
base_model = ResNet50(weights='imagenet',include_top=False,pooling = 'avg')

In [9]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 6 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [10]:
#base_model.layers.pop()

In [11]:
#base_model.outputs=base_model.layers[-1].output

In [49]:
x = base_model.output
#x = BatchNormalization()(x)
x = Dropout(0.8)(x)
pred = Dense(2,activation = 'softmax')(x)
model = Model(inputs=base_model.input,outputs=pred)

In [50]:
for layer in base_model.layers:layer.trainable=False

In [51]:
for layer in base_model.layers[-9:]:layer.trainable = True

In [52]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [16]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 6 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [33]:
model.optimizer.lr = 1e-3

In [67]:
#model.load_weights('resnet50-FT.h5')

In [30]:
#unfreeze -6: layer
model.fit_generator(train_generator,epochs=3,validation_data=valid_generator,
                    steps_per_epoch=trn_steps,validation_steps=val_steps,workers=5)

Epoch 1/3
180/180 [==============================] - 79s 440ms/step - loss: 0.0935 - acc: 0.9615 - val_loss: 0.0406 - val_acc: 0.9865
Epoch 2/3
180/180 [==============================] - 76s 422ms/step - loss: 0.0617 - acc: 0.9760 - val_loss: 0.0547 - val_acc: 0.9830
Epoch 3/3
180/180 [==============================] - 77s 427ms/step - loss: 0.0646 - acc: 0.9751 - val_loss: 0.0524 - val_acc: 0.9835


In [95]:
model.optimizer.lr = 1e-3

In [39]:
model.fit_generator(train_generator,epochs=3,validation_data=valid_generator,
                    steps_per_epoch=trn_steps,validation_steps=val_steps,workers=5)

Epoch 1/3
180/180 [==============================] - 79s 441ms/step - loss: 0.0690 - acc: 0.9751 - val_loss: 0.0448 - val_acc: 0.9840
Epoch 2/3
180/180 [==============================] - 76s 420ms/step - loss: 0.0394 - acc: 0.9850 - val_loss: 0.0289 - val_acc: 0.9895
Epoch 3/3
180/180 [==============================] - 76s 425ms/step - loss: 0.0363 - acc: 0.9861 - val_loss: 0.0430 - val_acc: 0.9865


In [40]:
model.save_weights('resnet50-FT-dropout.h5')

In [53]:
model.load_weights('resnet50-FT-dropout.h5')

In [54]:
model.optimizer.lr = 1e-4

In [55]:
model.fit_generator(train_generator,epochs=2,validation_data=valid_generator,
                    steps_per_epoch=trn_steps,validation_steps=val_steps,workers=5)

Epoch 1/2
180/180 [==============================] - 79s 441ms/step - loss: 0.0345 - acc: 0.9872 - val_loss: 0.0388 - val_acc: 0.9885
Epoch 2/2
180/180 [==============================] - 75s 418ms/step - loss: 0.0302 - acc: 0.9884 - val_loss: 0.0381 - val_acc: 0.9895


In [56]:
model.optimizer.lr = 1e-5

In [62]:
model.fit_generator(train_generator,epochs=2,validation_data=valid_generator,
                    steps_per_epoch=trn_steps,validation_steps=val_steps,workers=5)

Epoch 1/2
180/180 [==============================] - 79s 441ms/step - loss: 0.0334 - acc: 0.9880 - val_loss: 0.0347 - val_acc: 0.9905
Epoch 2/2
180/180 [==============================] - 76s 421ms/step - loss: 0.0333 - acc: 0.9875 - val_loss: 0.0418 - val_acc: 0.9890


In [107]:
model_json = model.to_json()

In [ ]:
#model.optimizer.lr = 1e-4

In [ ]:
#with augmentation

In [138]:
#model.fit_generator(train_generator,epochs=3,validation_data=valid_generator,steps_per_epoch=trn_steps,validation_steps=val_steps,workers=8)

Epoch 1/3
90/90 [==============================] - 69s 770ms/step - loss: 0.0686 - acc: 0.9764 - val_loss: 0.0744 - val_acc: 0.9820
Epoch 2/3
90/90 [==============================] - 66s 729ms/step - loss: 0.0438 - acc: 0.9834 - val_loss: 0.0839 - val_acc: 0.9810
Epoch 3/3
90/90 [==============================] - 66s 733ms/step - loss: 0.0423 - acc: 0.9833 - val_loss: 0.0882 - val_acc: 0.9800


In [ ]:
#trn_generator = get_data()

In [109]:
#trn_features = model.predict_generator(train_generator,steps = trn_steps,verbose=1)

24/90 [=======>......................] - ETA: 3:15

KeyboardInterrupt: 

In [187]:
#val_features = model.predict_generator(valid_generator,steps=val_steps,verbose=1)

8/8 [==============================] - 7s 842ms/step


In [110]:
test_generator = get_test_data(test_gen,test_path,input_size=224,batch_size=batch_size)

Found 12500 images belonging to 1 classes.


In [111]:
test_steps = comput_step(test_generator.n,batch_size)

In [112]:
test_features = model.predict_generator(test_generator,steps=test_steps,verbose=1)

49/49 [==============================] - 123s 3s/step


In [207]:
h.File('resnet50_FT1.h5').create_dataset('trn_features',data=trn_features)
h.File('resnet50_FT1.h5').create_dataset('val_features',data=val_features)

<HDF5 dataset "val_features": shape (2000, 2), type "<f4">

In [208]:
h.File('resnet50_FT1.h5').create_dataset('test_features',data=test_features)

<HDF5 dataset "test_features": shape (12500, 2), type "<f4">

In [209]:
h.File('resnet50_FT1.h5').create_dataset('val_label',data=valid_generator.classes)
h.File('resnet50_FT1.h5').create_dataset('trn_label',data=train_generator.classes)

<HDF5 dataset "trn_label": shape (23000,), type "<i4">

In [146]:
pred = test_features.clip(min=0.01,max=0.99)

In [147]:
filenames = test_generator.filenames

In [148]:
is_dog = pred[:,1]

In [149]:
index = [int(fname[fname.rfind('/')+1:fname.rfind('.')]) for fname in filenames]

In [150]:
sub_resnet50 = np.stack([index,is_dog],axis=1)

In [151]:
from pandas import DataFrame
import pandas as pd

In [152]:
submission_resnet50_FT_6train = DataFrame(sub_resnet50,columns=['id','label'])

In [153]:
submission_resnet50_FT_6train.id = submission_resnet50_FT_6train.id.astype(int)

In [154]:
submission_resnet50_FT_6train.to_csv('submission_resnet50_FT_6train.csv',index=None)

In [156]:
ls -l

total 2909808
-rw-rw-r-- 1 ec2-user ec2-user  219048328 Feb 10 12:36 InceptionResNetV2-FT.h5
-rwxrwx--- 1 ec2-user ec2-user     193342 Feb 10 18:21 InceptionResNetV2-FT.ipynb*
-rw-rw-r-- 1 ec2-user ec2-user     313718 Feb  5 17:34 pred.csv
drwxrwxr-x 2 ec2-user ec2-user       4096 Feb 26 08:08 __pycache__/
-rw-rw-r-- 1 ec2-user ec2-user   94783304 Feb 26 16:50 resnet50-FT-6train.h5
-rw-rw-r-- 1 ec2-user ec2-user   94783304 Feb 26 10:27 resnet50-FT.h5
-rw-rw-r-- 1 ec2-user ec2-user     109909 Feb 26 17:10 Resnet50-FT.ipynb
-rw-rw-r-- 1 ec2-user ec2-user     113903 Jun  8  2017 sample_submission.csv
-rw-rw-r-- 1 ec2-user ec2-user     313644 Feb  7 11:57 subm2-withBN.csv
-rw-rw-r-- 1 ec2-user ec2-user     377535 Feb  7 11:57 subm.csv
-rw-rw-r-- 1 ec2-user ec2-user     301716 Feb 26 17:09 submission_resnet50_FT_6train.csv
-rw-rw-r-- 1 ec2-user ec2-user     313810 Feb 10 18:17 submission_resnet50_FT.csv
drwxr-xr-x 3 ec2-user ec2-user     270336 Feb  5 17:01 test/
drwxr-xr-x 4 ec2-user ec2-u

In [145]:
rm submission_resnet50_FT_6train.csv